In [1]:
import pandas as pd 
import numpy as np
import os
from math import ceil
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
import re

In [2]:
import sys
sys.path.append('../evaluation/')

In [3]:
from utils_eval import build_pdb_dict

In [4]:
# root_path = 'pharmacophore_specific_eval/1ywn'  # 0 0 
# root_path = 'pharmacophore_specific_eval/4ara' # 0.48 0.49
# root_path = 'pharmacophore_specific_eval/1lhn'  # 0.08 0.08
# root_path = 'pharmacophore_specific_eval/4b84'  # 0.57 0.70
# root_path = 'pharmacophore_specific_eval/5ni6'  # 0.07 0.08
# root_path = 'pharmacophore_specific_eval/5idn'  # 0.23 0.25 
# root_path = 'pharmacophore_specific_eval/5ni2'  # 0.13 0.15
# root_path = 'pharmacophore_specific_eval/4ijw'  # 0.36 0.39
# root_path = 'pharmacophore_specific_eval/3ey4'  # 0.49 0.45
# root_path = 'pharmacophore_specific_eval/3r58'  # 0.70 0.70
# root_path = 'pharmacophore_specific_eval/4k1l'  # 0.58 0.62



# aromatic = True

In [20]:
# ligand_name = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0'  # 0 0
# ligand_name = '4b81_B_rec_4ara_c56_lig_tt_min_0'  # 0.41 0.51
# ligand_name = '1lhu_A_rec_1lhn_aon_lig_tt_min_0'  # 0.07 0.11
# ligand_name = '4b84_A_rec_4b84_z5k_lig_tt_min_0'  # 0.60 0.61
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'  
# ligand_name = '5hbh_A_rec_5idn_6a7_lig_tt_min_0'
# ligand_name = '5aen_A_rec_5ni2_dj3_lig_tt_min_0'
# ligand_name = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_3ey4_352_lig_tt_docked_12'
# ligand_name = '4h7c_A_rec_3r58_nps_lig_tt_min_0'  # 0.64 0.70
# ligand_name = '3ey4_C_rec_4k1l_sff_lig_tt_min_0'  # 0.59 0.59 

# ligand_name = '3hfg_A_rec_3hfg_17r_lig_tt_min_0'
# ligand_name = '3ey4_C_rec_5pgu_8k4_lig_tt_min_0'  # 0.11 0.10
# ligand_name = '5ni2_A_rec_5aen_dp8_lig_tt_min_0'
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_2ilt_nn1_lig_tt_min_0'  # 0.16 0.20 
# ligand_name = '4bnn_A_rec_4bnm_tcu_lig_tt_min_0'
# ligand_name = '4yvx_A_rec_1s2a_imn_lig_tt_min_0'
# ligand_name = '3czr_B_rec_4bb5_hd2_lig_tt_min_0'
# ligand_name = '3czr_A_rec_2ilt_nn1_lig_tt_min_0'
# ligand_name = '4b84_A_rec_4btl_5gz_lig_tt_min_0'
# ligand_name = '3fue_A_rec_3ftv_11x_lig_tt_min_0'  # 0.04 0.05
# ligand_name = '4bnk_B_rec_4d44_ja3_lig_tt_min_0'  # 0.26 0.27
# ligand_name = '3qqp_B_rec_4k1l_sff_lig_tt_min_0'
# ligand_name = '4wdu_A_rec_3r94_flr_lig_tt_min_0'  # 0.37 0.27
# ligand_name = '4alj_A_rec_4bnf_pv4_lig_tt_docked_0'  # 0.24 0.24
# ligand_name = '1ejj_A_rec_1o99_2pg_lig_tt_docked_0'
# ligand_name = '4h96_B_rec_4h95_06u_lig_tt_min_0'  # 0.40 0.42
# ligand_name = '2oa6_D_rec_4kvi_1sv_lig_tt_min_0'  # 0.36 0.41
# ligand_name = '4hx5_B_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgy_D_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgv_A_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '4kux_B_rec_4kvw_jf4_lig_tt_min_0'  # 0.61 0.63
# ligand_name = '4bnk_B_rec_4bnl_w1g_lig_tt_min_0'
# ligand_name = '4b82_B_rec_4ara_c56_lig_tt_min_0'  # 0.42 0.53
# ligand_name = '4pm0_A_rec_4y2b_epk_lig_tt_min_0'
# ligand_name = '4ms6_A_rec_3fuh_5h1_lig_tt_min_0'  # 0.02 0.02
# ligand_name = '5ivg_B_rec_4kvy_jf1_lig_tt_docked_9'  # 0.61 0.69
# ligand_name = '3r8h_A_rec_3r8h_zom_lig_tt_min_0'
# ligand_name = '1aoe_A_rec_3qlr_qlr_lig_tt_docked_4'  # 0.19 0.28
# ligand_name = '2ilt_A_rec_3ey4_352_lig_tt_min_0'  # 0.70 0.71
# ligand_name = '2g1q_A_rec_1x88_nat_lig_tt_min_0'  # 0.59 0.59
# ligand_name = '5imp_B_rec_4kwd_jf2_lig_tt_min_0'  # 0.54 0.55
# ligand_name = '4d45_A_rec_4ali_tcl_lig_tt_min_0'  # 0.57 0.52
# ligand_name = '5ivg_B_rec_4kvw_jf4_lig_tt_min_0'
# ligand_name = '2rbe_C_rec_3byz_h11_lig_tt_min_0'
# ligand_name = '2whr_A_rec_3zlv_hi6_lig_tt_min_0'
# ligand_name = '4hmn_A_rec_3ufy_npx_lig_tt_min_0'
# ligand_name = '4h7c_A_rec_3r6i_jms_lig_tt_min_0'
# ligand_name = '1pha_A_rec_1phg_myt_lig_tt_min_0'
# ligand_name = '1phd_A_rec_1phg_myt_lig_tt_min_0'
# ligand_name = '1phg_A_rec_1phe_piy_lig_tt_min_0'
# ligand_name = '1phg_A_rec_1phg_myt_lig_tt_min_0'
# ligand_name = '1p2y_A_rec_1phg_myt_lig_tt_min_0'
# ligand_name = '1p2y_A_rec_1phe_piy_lig_tt_min_0'
# ligand_name = '5lgq_D_rec_4ikp_4ik_lig_tt_docked_1'
# ligand_name = '1c8j_A_rec_1p2y_nct_lig_tt_min_0'
# ligand_name = '1l5l_A_rec_1l4l_xyd_lig_tt_min_0'
# ligand_name = '4l0s_A_rec_5al5_jl9_lig_tt_min_0'
# ligand_name = '2pg2_A_rec_2wog_zzd_lig_tt_min_0'
# ligand_name = '4yur_A_rec_3cok_anp_lig_tt_min_0'
# ligand_name = '3ny8_A_rec_4ldo_ale_lig_tt_docked_4'  # remove atom 11

# ligand_name = '4mrg_A_rec_5bzf_4x3_lig_tt_docked_2'
# ligand_name = '1k4h_A_rec_1p0b_pq0_lig_tt_docked_0'

# ligand_name = '3nbq_B_rec_3nbq_urf_lig_tt_docked_2'
# ligand_name = '3p1c_B_rec_3p1f_3pf_lig_tt_min_0'
# ligand_name = '5swg_A_rec_5swt_71a_lig_tt_min_0'
# ligand_name = '3euf_A_rec_3nbq_urf_lig_tt_min_0'
# ligand_name = '2vce_A_rec_2vce_tc7_lig_tt_docked_0'
# ligand_name = '4xud_A_rec_5lsa_dnc_lig_tt_docked_2'
# ligand_name = '1eoc_A_rec_2buz_4nc_lig_tt_docked_4'
# ligand_name = '1y5x_A_rec_4puk_2wu_lig_tt_min_0'
ligand_name = '2bur_A_rec_1eoc_4nc_lig_tt_min_0'
# ligand_name = '1k4h_A_rec_1p0b_pq0_lig_tt_docked_0'
# ligand_name = '1eob_A_rec_2buz_4nc_lig_tt_min_0'
# ligand_name = '4mrg_A_rec_5bzf_4x3_lig_tt_docked_2'
# ligand_name = '4xuc_A_rec_5lsa_dnc_lig_tt_min_0'
# ligand_name = '1kwc_B_rec_1eir_bpy_lig_tt_min_0'
# ligand_name = '2vce_A_rec_2vce_tc7_lig_tt_min_0'
# ligand_name = '1eoc_A_rec_1eoc_4nc_lig_tt_min_0'
# ligand_name = '5mpz_A_rec_5mqg_f31_lig_tt_docked_3'
# ligand_name = '2buq_A_rec_2buz_4nc_lig_tt_min_0'
# ligand_name = '4mrf_A_rec_4mre_2c9_lig_tt_min_0'
# ligand_name = '1k9s_D_rec_1a69_fmb_lig_tt_min_0'
# ligand_name = '5i00_A_rec_1enu_apz_lig_tt_min_0'
# ligand_name = '1eoc_A_rec_2buz_4nc_lig_tt_min_0'
# ligand_name = '2bur_A_rec_2buz_4nc_lig_tt_min_0'
# ligand_name = '3eue_A_rec_3nbq_urf_lig_tt_min_0'
# ligand_name = '4xue_B_rec_5lsa_dnc_lig_tt_docked_10'
# ligand_name = '5jxq_A_rec_1p0b_pq0_lig_tt_min_0'
# ligand_name = '1eob_A_rec_2buu_4nc_lig_tt_min_0'
# ligand_name = '3rr4_A_rec_1p0b_pq0_lig_tt_min_0'
# ligand_name = '2vch_A_rec_2vce_tc7_lig_tt_min_0'
# ligand_name = '2buu_A_rec_1eoc_4nc_lig_tt_min_0'
# ligand_name = '4xuc_A_rec_5lsa_dnc_lig_tt_docked_7'

# ligand_name = '5ehq_A_rec_1j06_ae3_lig_tt_min_0'

# folder_name = ligand_name[ligand_name.rfind('rec')+4:ligand_name.rfind('rec')+8]
bridge_type = 'vp'
root_path = os.path.join('structure_based', bridge_type, ligand_name)

aromatic = True

In [21]:
log_path = os.path.join(root_path, 'logs')
log_path = log_path+'_aromatic' if aromatic else log_path

raw_data_path = '../../data/cleaned_crossdocked_data/raw'

log_files = os.listdir(log_path)
pdb_dict, pdb_rev_dict = build_pdb_dict(raw_data_path)

In [22]:
def get_minimized_affinity(filename, mode='ref'):
    if mode == 'ref':
        with open(filename, 'r') as file:
            for line in file:
                if '<minimizedAffinity>' in line:
                    # The next line after '<minimizedAffinity>' contains the value
                    return float(next(file))
            
    else:
        with open(filename, 'r') as f:
            lines = f.readlines()
            lines = lines[19:28]
#             print(len(lines))
            n = len(lines)
            arr = np.zeros(n)
            cnn_arr = np.zeros(n)
            # print(filename)
            for i in range(n):
                arr[i] = float(lines[i][11:17])
                cnn_arr[i] = float(lines[i][34:42])
#             print(arr)
            try:
                score = np.min(arr)
                cnn_score = np.min(cnn_arr)
            except:
                score = None
                cnn_score = None
                print(f'{filename} failed')
                print('its scores: ', arr)
        return score, cnn_score

In [23]:
def get_affinity_best_pose(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        lines = lines[19:28]
        try:
            score = float(lines[0][11:17])
            cnn_score = float(lines[0][34:42])
            # cnn_score = float(lines[0][24:30])
        except:
            score = None
            cnn_score = None
        return score, cnn_score

In [33]:
# ligand_filename = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0.sdf'
# ligand_filename = '4b81_B_rec_4ara_c56_lig_tt_min_0.sdf'
# ligand_filename = '1lhu_A_rec_1lhn_aon_lig_tt_min_0.sdf'
# ligand_filename = '4b84_A_rec_4b84_z5k_lig_tt_min_0.sdf'
# ligand_filename = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0.sdf'
# ligand_filename = '5hbh_A_rec_5idn_6a7_lig_tt_min_0.sdf'
# ligand_filename = '5aen_A_rec_5ni2_dj3_lig_tt_min_0.sdf'
# ligand_filename = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0.sdf'
# ligand_filename = '4yyz_B_rec_3ey4_352_lig_tt_docked_12.sdf'
# ligand_filename = '4h7c_A_rec_3r58_nps_lig_tt_min_0.sdf'
# ligand_filename = '3ey4_C_rec_4k1l_sff_lig_tt_min_0.sdf'

ligand_filename = ligand_name + '.sdf'
pdb_folder = pdb_rev_dict[ligand_filename]
ref_file = os.path.join(raw_data_path, pdb_folder, ligand_filename)
ref_score = get_minimized_affinity(ref_file, 'ref')

score_dict = {}
for log_file in tqdm(log_files):
    filename = log_file.split('.')[0]
#     print(log_file)
    # gen_score, cnn_score = get_minimized_affinity(os.path.join(log_path, log_file), 'gen')
    gen_score, cnn_score = get_affinity_best_pose(os.path.join(log_path, log_file))
    score_dict[filename] = [gen_score, cnn_score, ref_score]

100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 22262.76it/s]


In [34]:
df = pd.DataFrame.from_dict(score_dict, orient='index', columns=['Binding affinity', 'CNN score', 'Reference'])
df

,Binding affinity,CNN score,Reference
1eoc_0,-6.45,0.9194,-5.39065
1eoc_5,-6.47,0.9232,-5.39065
1eoc_6,-6.47,0.9186,-5.39065
1eoc_4,-6.49,0.9162,-5.39065
1eoc_3,-6.40,0.9220,-5.39065
1eoc_1,-6.49,0.9208,-5.39065
1eoc_7,-6.49,0.9254,-5.39065
1eoc_2,-6.49,0.9240,-5.39065
1eoc_8,-6.46,0.9198,-5.39065
1eoc_9,-6.48,0.9209,-5.39065


In [35]:
filtered_df = df.loc[df['Binding affinity']<df['Reference']]
filtered_df

,Binding affinity,CNN score,Reference
1eoc_0,-6.45,0.9194,-5.39065
1eoc_5,-6.47,0.9232,-5.39065
1eoc_6,-6.47,0.9186,-5.39065
1eoc_4,-6.49,0.9162,-5.39065
1eoc_3,-6.40,0.9220,-5.39065
1eoc_1,-6.49,0.9208,-5.39065
1eoc_7,-6.49,0.9254,-5.39065
1eoc_2,-6.49,0.9240,-5.39065
1eoc_8,-6.46,0.9198,-5.39065
1eoc_9,-6.48,0.9209,-5.39065


In [36]:
sorted_df = filtered_df.sort_values(by='CNN score', ascending=False)
sorted_df

,Binding affinity,CNN score,Reference
1eoc_7,-6.49,0.9254,-5.39065
1eoc_2,-6.49,0.9240,-5.39065
1eoc_5,-6.47,0.9232,-5.39065
1eoc_3,-6.40,0.9220,-5.39065
1eoc_9,-6.48,0.9209,-5.39065
1eoc_1,-6.49,0.9208,-5.39065
1eoc_8,-6.46,0.9198,-5.39065
1eoc_0,-6.45,0.9194,-5.39065
1eoc_6,-6.47,0.9186,-5.39065
1eoc_4,-6.49,0.9162,-5.39065


In [37]:
ba_res_filename = os.path.join(root_path, 'binding_affinity')
ba_res_filename = ba_res_filename+'_aromatic.csv' if aromatic else ba_res_filename+'.csv'

# sorted_df.to_csv(ba_res_filename)
# ba_res_filename

In [38]:
pp_match_csv = os.path.join(root_path, 'aromatic_scores.csv' if aromatic else 'basic_scores.csv')
pp_match_df = pd.read_csv(pp_match_csv, header=0, index_col=0)
pp_match_df = pp_match_df.rename(columns = {'score':'PP match score'})
pp_match_df

,PP match score
1eoc_8,1.0
1eoc_9,1.0
1eoc_7,1.0
1eoc_4,1.0
1eoc_1,1.0
1eoc_5,1.0
1eoc_6,1.0
1eoc_3,1.0
1eoc_2,1.0
1eoc_0,1.0


In [39]:
joint_df = df.join(pp_match_df)
joint_df

,Binding affinity,CNN score,Reference,PP match score
1eoc_0,-6.45,0.9194,-5.39065,1.0
1eoc_5,-6.47,0.9232,-5.39065,1.0
1eoc_6,-6.47,0.9186,-5.39065,1.0
1eoc_4,-6.49,0.9162,-5.39065,1.0
1eoc_3,-6.40,0.9220,-5.39065,1.0
1eoc_1,-6.49,0.9208,-5.39065,1.0
1eoc_7,-6.49,0.9254,-5.39065,1.0
1eoc_2,-6.49,0.9240,-5.39065,1.0
1eoc_8,-6.46,0.9198,-5.39065,1.0
1eoc_9,-6.48,0.9209,-5.39065,1.0


In [40]:
joint_res_fn = 'joint_res_aromatic.csv' if aromatic else 'joint_res.csv'
joint_df.to_csv(os.path.join(root_path, joint_res_fn))

In [41]:
sum(df['Binding affinity'] <= df['Reference']) / len(df)

1.0